### Phase 2 - Data Preparation & Transformation (Flipkart Smartphone Dataset)

**Project Objective**  

The goal of this phase is to transform the raw scraped data into a clean, structured, and analysis-ready format for both business visualization in Power BI and machine learning-based price prediction.

Rather than performing an extensive EDA, this phase emphasizes building a reliable and standardized dataset through the following key steps:

- Assess and enhance overall data quality
- Handle missing values, inconsistencies, and duplicates
- Standardize and normalize technical features (RAM, ROM, Display, Battery, Processor, Camera)
- Engineer meaningful variables to strengthen model accuracy
- Cap or remove outliers for price consistency
- Structure the final dataset for seamless integration with Power BI and training predictive models

This ensures that the dataset is clean, consistent, and ready to generate both analytical insights and predictive intelligence.

---

Let’s set the stage by importing the essential libraries.

In [148]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

### 1. Data Loading & Initial Overview

Let’s take a first step by loading the CSV file.

In [149]:
df = pd.read_csv('flipkart_smartphones.csv')

Let’s take a quick look at the first few rows of our dataset.

In [150]:
df.head()

,Name,Price,Discount,Rating,Specs
0,"Samsung Galaxy S24 5G Snapdragon (Onyx Black, ...",39999,46% off,4.6,8 GB RAM | 128 GB ROM; 15.75 cm (6.2 inch) Ful...
1,"Nothing Phone (3a) Pro (Grey, 256 GB)",29999,14% off,4.5,8 GB RAM | 256 GB ROM; 43.66 cm (17.19 cm) Ful...
2,"realme 15x 5G (Marine Blue, 128 GB)",16999,10% off,4.3,6 GB RAM | 128 GB ROM | Expandable Upto 2 TB; ...
3,POCO C71 - Locked with Airtel Prepaid (Desert...,5599,37% off,3.9,4 GB RAM | 64 GB ROM | Expandable Upto 2 TB; 1...
4,"Nothing Phone (3) (White, 512 GB)",89999,5% off,4.4,16 GB RAM | 512 GB ROM; 16.94 cm (6.67 inch) D...


Let’s explore the dataset’s structure.

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      972 non-null    object 
 1   Price     972 non-null    int64  
 2   Discount  930 non-null    object 
 3   Rating    970 non-null    float64
 4   Specs     972 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 38.1+ KB


Let’s check for missing values in the dataset.

In [152]:
df.isnull().sum()

Name         0
Price        0
Discount    42
Rating       2
Specs        0
dtype: int64

Let’s Handle Missing Values to Ensure Data Completeness

In [153]:
# 1. Fill missing Discount with 0 (since it means no discount info given)
df['Discount'] = df['Discount'].fillna(0)

# 2. Fill missing Rating with brand-wise mean, then global mean if still null
df['Rating'] = df.groupby('Name')['Rating'].transform(lambda x: x.fillna(x.mean()))
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())

print("Null values in Discount and Rating handled successfully.")

Null values in Discount and Rating handled successfully.


### 2. Feature Extraction & Data Structuring 

Let’s Extract and Standardize Smartphone Brands for Consistency

In [154]:
# --- Brand Extraction Logic ---

# Define a comprehensive list of known brands and common typos for robust matching.
valid_brands = [
    'samsung', 'nothing', 'realme', 'poco', 'motorola', 'oppo', 'vivo',
    'apple', 'google', 'redmi', 'infinix', 'iqoo', 'oneplus', 'tecno',
    'itel', 'lava', 'cmf', 'micromax', 'nokia', 'honor', 'lyf',
    'karbonn', 'gionee', 'xolo', 'panasonic', 'hmd', 'ai+', 'idea',
    'snexian', 'snectian', # Handle data typo
    'sia', 'siavantage',
    'i kall', 'nextbit', 'coolpad', 'alcatel',
    'hotline', 'gfive', 'nuveck'
]

# Dictionary to map variations or sub-brands to a standard brand name.
brand_mapping = {
    'mi ': 'Redmi',
    'xiaomi ': 'Redmi',
    'cmf by nothing': 'Nothing'
}

# Function to extract a standardized brand name from the product title.
def extract_brand_final(name):
    # Standardize name to lowercase for case-insensitive search.
    name_lower = str(name).lower()

    # Prioritize special mappings (e.g., 'mi' -> 'Redmi').
    for key, brand in brand_mapping.items():
        if key in name_lower:
            return brand

    # Iterate through the primary list to find a match at the start of the string.
    for brand in valid_brands:
        if name_lower.startswith(brand):
            # Handle multi-word brands for correct capitalization.
            if brand == 'i kall':
                return 'I Kall'
            return brand.title()
            
    # Return 'Unknown' if no brand is matched.
    return 'Unknown'

# Apply the function to the 'Name' column to generate the 'Brand' feature.
df['Brand'] = df['Name'].apply(extract_brand_final)


# --- Validation Step ---

# Review the distribution of the extracted brands.
print("Distribution of extracted brands:")
print(df['Brand'].value_counts())

# Isolate and review any remaining unclassified entries.
print("\nProducts with unclassified brands:")
unknown_df = df[df['Brand'] == 'Unknown']

if unknown_df.empty:
    print("Success: All brands have been classified.")
else:
    # Display the names of any products that were not classified.
    print(unknown_df['Name'].to_list())

Distribution of extracted brands:
Brand
Nothing     243
Samsung     196
Realme      146
Redmi        61
Poco         54
Motorola     43
Vivo         31
Tecno        30
Oneplus      27
Infinix      24
Oppo         23
Iqoo         20
Apple        17
Google       15
Ai+          10
Lava          9
Alcatel       6
Itel          5
Xolo          3
Nokia         2
Idea          1
Unknown       1
Sia           1
Coolpad       1
Nextbit       1
Karbonn       1
Gionee        1
Name: count, dtype: int64

Products with unclassified brands:
['(Refurbished) MOTOROLA G34 5G (Ocean Green, 128 GB)']


- This block separate the brand name from the product title and validates it against a curated list of known smartphone brands.
If the first word is invalid, the logic checks the second word, otherwise assigns “Unknown”. This ensures brand-level analysis in Power BI is accurate, clean, and free from noisy or inconsistent entries.

Let’s extracting Key Smartphone Features for Structured Analysis

In [155]:
# --- Helper Function for RAM/ROM Normalization ---

def normalize_memory(specs_series, memory_type):
    """Extracts and normalizes RAM or ROM to GB, handling both GB and MB units."""
    pattern = fr"(\d+)\s*(GB|MB)\s*{memory_type}"
    extracted = specs_series.str.extract(pattern, expand=True)
    extracted.columns = ['value', 'unit']
    
    extracted['value'] = pd.to_numeric(extracted['value'], errors='coerce')
    
    # Standardize to GB
    is_mb = extracted['unit'].str.upper() == 'MB'
    extracted.loc[is_mb, 'value'] = extracted.loc[is_mb, 'value'] / 1024
    
    return extracted['value']


# --- Feature Extraction Block ---

# 1. Extract Phone Model
df['Phone_Model'] = df['Name'].str.split(r'\(').str[0].str.strip()

# 2. Clean and convert Discount column
df["Discount"] = pd.to_numeric(
    df["Discount"]
    .astype(str)
    .str.replace("% off", "", regex=False)
    .str.strip(), 
    errors="coerce"
)

# 3. Calculate Original Price from current price and discount
df["Original_Price"] = df.apply(
    lambda row: round(row["Price"] / (1 - (row["Discount"] / 100)), 2)
    if pd.notna(row["Price"]) and pd.notna(row["Discount"]) and row["Discount"] > 0
    else row["Price"],
    axis=1
)

# 4. Extract and Normalize RAM & ROM
df["RAM_GB"] = normalize_memory(df["Specs"], 'RAM')
df["ROM_GB"] = normalize_memory(df["Specs"], 'ROM')


# 5. Extract Display Features
df["Display_Size_inch"] = pd.to_numeric(df["Specs"].str.extract(r'([\d\.]+)\s*inch', expand=False), errors='coerce')
display_pattern = r"(Super Retina|Full HD\+|FHD\+|Full HD|HD\+|HD|Super AMOLED|AMOLED|OLED|Retina)"
df["Display_Type"] = df["Specs"].str.extract(display_pattern, expand=False)

# 6. Extract Battery Capacity
df["Battery_mAh"] = pd.to_numeric(df["Specs"].str.extract(r"(\d+)\s*mAh", expand=False), errors='coerce')

# 7. Extract Processor Information
processor_pattern = r";\s*([^;]+?Processor)"
df["Processor"] = df["Specs"].str.extract(processor_pattern, expand=False).str.strip()
# Fallback pattern for cases where the primary one fails
fallback_pattern = r"([A-Za-z0-9\s.-]+?Processor)"
df["Processor"].fillna(df["Specs"].str.extract(fallback_pattern, expand=False), inplace=True)

# 8. Extract Warranty Information
df["Warranty_Years"] = pd.to_numeric(df["Specs"].str.extract(r"(\d+)\s*Year", expand=False), errors='coerce')

print("Feature extraction complete. All new columns have been successfully added.")

Feature extraction complete. All new columns have been successfully added.


C:\Users\HETVI\AppData\Local\Temp\ipykernel_19320\3757091295.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Processor"].fillna(df["Specs"].str.extract(fallback_pattern, expand=False), inplace=True)


- This code block extracts and organizes key smartphone attributes such as RAM, ROM, display size, display type, cameras, battery, and processor. By transforming raw, unstructured text into clean numeric and categorical columns, it prepares the dataset for reliable comparisons, trend analysis, and professional visualizations in tools like Power BI.

In [156]:


# --- 1. Data Pre-processing (Essential for Processor Accuracy) ---
# English Comment: Define a function to clean the 'Specs' column.
def clean_specs(spec_string):
    if not isinstance(spec_string, str):
        return ""
    # Replace newlines with a space to handle multi-line processor descriptions.
    cleaned = spec_string.replace('\n', ' ').replace('\r', ' ')
    # Standardize spacing around semicolons.
    cleaned = re.sub(r'\s*;\s*', '; ', cleaned)
    return cleaned

# English Comment: Apply the cleaning function.
df['Specs_Clean'] = df['Specs'].apply(clean_specs)

# --- 2. Processor Extraction ---
# English Comment: First pattern tries to find the processor after a semicolon.
processor_pattern = r";\s*([^;]+?Processor)"
df["Processor"] = df["Specs_Clean"].str.extract(processor_pattern, expand=False).str.strip()

# English Comment: Fallback pattern for cases where the first one fails.
fallback_pattern = r"([A-Za-z0-9\s.-]+?Processor)"

# English Comment: Fill any missing values using the fallback and fix the FutureWarning.
df["Processor"] = df["Processor"].fillna(df["Specs_Clean"].str.extract(fallback_pattern, expand=False).str.strip())

# English Comment: Drop the temporary clean column.
df.drop(columns=['Specs_Clean'], inplace=True)

print("Processor extraction complete.")

# English Comment: Display the 'Name' and new 'Processor' column to verify.
print("\nVerification of extracted 'Processor' column:")
print(df[['Name', 'Processor']].head(10))


Processor extraction complete.

Verification of extracted 'Processor' column:
                                                Name  \
0  Samsung Galaxy S24 5G Snapdragon (Onyx Black, ...   
1              Nothing Phone (3a) Pro (Grey, 256 GB)   
2                realme 15x 5G (Marine Blue, 128 GB)   
3  POCO C71  - Locked with Airtel Prepaid (Desert...   
4                  Nothing Phone (3) (White, 512 GB)   
5                  Nothing Phone (3) (Black, 256 GB)   
6                   realme C61 (Marble Black, 64 GB)   
7                       POCO C71 (Cool Blue, 128 GB)   
8                            Ai+ Pulse (Blue, 64 GB)   
9                   realme C61 (Safari Green, 64 GB)   

                                           Processor  
0                                  8 Gen 3 Processor  
1                                  7s Gen3 Processor  
2     Dimensity 6300 6nm Octa-core chipset Processor  
3  Unisoc T7250 Max clock speed:  2 x A75@1.8GHz ...  
4                 8s Gen 4 Mob

Let’s Organize Columns for a Clean, Business-Ready Dataset

In [157]:
# Define the ideal column order for structured output
ideal_order = [
    "Brand",
    "Phone_Model",
    "Name",
    "Price",
    "Original_Price",
    "Discount",
    "Rating",
    "RAM_GB",
    "ROM_GB",
    "Processor",
    "Display_Size_inch",
    "Display_Type",
    "RearCam",
    "FrontCam",
    "Battery_mAh",
    "Processor",
    "Warranty_Years",
    "Specs"
]

# Reorder dataframe columns while ignoring any missing ones
df = df[[col for col in ideal_order if col in df.columns]]

# Save the organized dataset
output_file = "flipkart_smartphones_organized.csv"
df.to_csv(output_file, index=False)

print(f"Columns reordered & saved as '{output_file}'")


Columns reordered & saved as 'flipkart_smartphones_organized.csv'


- This block rearranges the dataset into an ideal column order for better readability and structured analysis.
It then saves the final version as a clean CSV file, ensuring the dataset is well-organized, consistent, and ready for Power BI integration.

### 3. Data Cleaning & Preprocessing

Let’s Reload the Final Organized Dataset

In [158]:
new_df = pd.read_csv("flipkart_smartphones_organized.csv")

Let’s Inspect the Dataset Structure and Data Types

In [159]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              972 non-null    object 
 1   Phone_Model        971 non-null    object 
 2   Name               972 non-null    object 
 3   Price              972 non-null    int64  
 4   Original_Price     972 non-null    float64
 5   Discount           972 non-null    int64  
 6   Rating             972 non-null    float64
 7   RAM_GB             954 non-null    float64
 8   ROM_GB             971 non-null    float64
 9   Processor          871 non-null    object 
 10  Display_Size_inch  849 non-null    float64
 11  Display_Type       586 non-null    object 
 12  Battery_mAh        955 non-null    float64
 13  Processor.1        871 non-null    object 
 14  Warranty_Years     919 non-null    float64
 15  Specs              972 non-null    object 
dtypes: float64(7), int64(2), o

Let’s Rechecking for missing values in the dataset again.

In [160]:
new_df.isnull().sum()

Brand                  0
Phone_Model            1
Name                   0
Price                  0
Original_Price         0
Discount               0
Rating                 0
RAM_GB                18
ROM_GB                 1
Processor            101
Display_Size_inch    123
Display_Type         386
Battery_mAh           17
Processor.1          101
Warranty_Years        53
Specs                  0
dtype: int64

Let's Impute Missing Data for RAM, ROM, Display, Camera, Battery, and Processor

In [161]:
# 1. RAM_GB - Impute with brand-level median, then global median
df['RAM_GB'] = df.groupby('Brand')['RAM_GB'].transform(lambda x: x.fillna(x.median()))
df['RAM_GB'] = df['RAM_GB'].fillna(df['RAM_GB'].median())

# 2. ROM_GB - Impute with brand-level median, then global median
df['ROM_GB'] = df.groupby('Brand')['ROM_GB'].transform(lambda x: x.fillna(x.median()))
df['ROM_GB'] = df['ROM_GB'].fillna(df['ROM_GB'].median())

# 3. Display_Size_inch - Impute with brand-level mean, then global mean
df['Display_Size_inch'] = df.groupby('Brand')['Display_Size_inch'].transform(lambda x: x.fillna(x.mean()))
df['Display_Size_inch'] = df['Display_Size_inch'].fillna(df['Display_Size_inch'].mean())

# 4. Display_Type (IMPROVED) - Impute with brand-level mode, then global mode
# This handles cases where a brand might not have a mode.
df['Display_Type'] = df.groupby('Brand')['Display_Type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown"))
df['Display_Type'] = df['Display_Type'].fillna(df['Display_Type'].mode()[0]) # Global fallback

# 5. Battery_mAh - Impute with brand-level median, then global median
df['Battery_mAh'] = df.groupby('Brand')['Battery_mAh'].transform(lambda x: x.fillna(x.median()))
df['Battery_mAh'] = df['Battery_mAh'].fillna(df['Battery_mAh'].median())

# 6. Processor - Impute with a placeholder string
df['Processor'] = df['Processor'].fillna('Not Specified')

# 7. Warranty_Years - Impute with 0
df['Warranty_Years'] = df['Warranty_Years'].fillna(0)

print("Missing data imputation complete!")

c:\Users\HETVI\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\HETVI\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\HETVI\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Missing data imputation complete!


c:\Users\HETVI\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


- This block filled missing values with brand-wise median/mean (with global fallback) for RAM, ROM, Display Size, Battery.
Categorical fields (Display Type, Camera, Processor) were filled with mode or default labels.

In [162]:
df.isnull().sum()

Brand                0
Phone_Model          0
Name                 0
Price                0
Original_Price       0
Discount             0
Rating               0
RAM_GB               0
ROM_GB               0
Processor            0
Display_Size_inch    0
Display_Type         0
Battery_mAh          0
Processor            0
Warranty_Years       0
Specs                0
dtype: int64

Let's View the Statistical Summary of Numeric Columns

In [163]:
df.describe().T 

,count,mean,std,min,25%,50%,75%,max
Price,972.0,31426.389918,24699.682817,4997.0,13999.00000,24999.000,39999.000,135499.00
Original_Price,972.0,40535.974640,28068.115258,5149.0,18887.78000,32940.000,74072.220,163788.06
Discount,972.0,23.541152,14.468665,0.0,13.00000,22.000,35.000,55.00
Rating,972.0,4.383814,0.174879,3.0,4.30000,4.400,4.500,4.90
RAM_GB,972.0,8.233539,3.061334,1.0,6.00000,8.000,8.000,16.00
ROM_GB,972.0,187.629630,112.128929,4.0,128.00000,128.000,256.000,512.00
Display_Size_inch,972.0,6.611919,0.318214,3.0,6.64275,6.676,6.745,7.82
Battery_mAh,972.0,5261.206790,855.243644,1300.0,5000.00000,5000.000,5500.000,7550.00
Warranty_Years,972.0,0.946502,0.229669,0.0,1.00000,1.000,1.000,2.00


Let's View the Summary of categoriacl Columns

In [164]:
df.describe(include='object').T 

,count,unique,top,freq
Brand,972,27,Nothing,243
Phone_Model,972,226,Nothing Phone,237
Name,972,493,"Samsung Galaxy S24 5G Snapdragon (Onyx Black, ...",41
Processor,972,139,7s Gen3 Processor,123
Display_Type,972,7,Full HD+,747
Processor,972,139,7s Gen3 Processor,123
Specs,972,358,8 GB RAM | 128 GB ROM; 15.75 cm (6.2 inch) Ful...,98


Let's Identify Outliers in All Numeric Features Using the IQR Method

In [165]:
num_col = df.select_dtypes(include=['float64', 'int64']).columns

for col in num_col:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col}: {outliers.shape[0]} outliers")

Price: 129 outliers
Original_Price: 1 outliers
Discount: 0 outliers
Rating: 25 outliers
RAM_GB: 223 outliers
ROM_GB: 71 outliers
Display_Size_inch: 173 outliers
Battery_mAh: 235 outliers
Warranty_Years: 54 outliers


Let's Cap Outliers in Numeric Columns Using the IQR Method

In [166]:
# for col in num_col:

#     Q1 = df[col].quantile(0.25)
#     Q3 = df[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower = Q1 - 1.5 * IQR
#     upper = Q3 + 1.5 * IQR

#     # Count outliers
#     outliers = df[(df[col] < lower) | (df[col] > upper)]
#     print(f"{col}: {outliers.shape[0]} outliers capped")

#     # Create capped version
#     capped_col = f"{col}_iqr_capped"
#     df[capped_col] = df[col].clip(lower=lower, upper=upper)
    
#     fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
#     sns.boxplot(x=df[col], ax=axes[0], color='#C39898')
#     axes[0].set_title(f"Before Capping: {col}")
#     sns.boxplot(x=df[capped_col], ax=axes[1], color='#DBB5B5')
#     axes[1].set_title(f"After IQR Capping: {capped_col}")
#     plt.tight_layout()
#     plt.show()
    
#     df[col] = df[capped_col]
#     df.drop(columns=[capped_col], inplace=True)

- Identified outliers in numeric columns using the IQR method.
- Capped outliers to lower and upper IQR bounds to reduce their impact.
- Visualized before and after capping with boxplots for better understanding.

---

Let's save the cleaned dataset for visualization.

In [168]:
df.to_csv("flipkart_smartphones_cleaned.csv", index=False)

<!-- --- -->

### Phase 3 Conclusion & Next Steps  

- We successfully turned the messy, raw data into a clean and organized dataset.
- First, we pulled out all the important details like RAM, Processor, and Brand from the text and put them into their own separate, clean columns.
- Next, we smartly filled in any missing information and fixed the outlier values (like unusually high prices) so they won't skew the analysis.
- Finally, we arranged everything into a logical order and saved the file. The dataset is now fully prepared and ready for you to use in Power BI or for a machine learning project.
 

---

### Next Steps (Phase 4 – Machine Learning Model)

The cleaned dataset is now ready to be used to build a predictive machine learning model. The key objectives are to:

- Predict Smartphone Prices - Train a regression model to accurately estimate a phone's price based on its features like brand, RAM, and battery capacity.

- Identify Key Price Drivers - Use feature importance analysis from the model to determine which specifications have the most significant impact on a phone's market price.

- Discover Market Segments - Apply clustering algorithms to automatically group similar phones, revealing distinct market segments such as "budget camera phones" or "premium flagships."

- Inform Pricing Strategy - Leverage the model’s predictions and insights to support data-driven decisions on competitive positioning and pricing.

---
---